<a href="https://colab.research.google.com/github/juancuevas-ops/inteligencia-artificial/blob/main/Personas_vacunadas_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Paso 1: configurar el registro local
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Paso 2: encuentra columnas en el marco de datos
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Paso 3: conéctese a un archivo DB si existe, de lo contrario, cree un nuevo archivo
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Crear una Tabla
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: actualizar el  dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Confirmar los cambios y cerrar la conexión.
    con.commit()
    con.close()

In [2]:
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connectando a  SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Ejecutando la consulta  Sql
    cursor = con.execute(sql_query_string)

    # Step 3: Obtener los datos y los nombres de las columnas
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: cerramos la conexion
    con.close()

    # Step 5: Devolver como un marco de datos
    return pd.DataFrame(result_data, columns=cols)

In [3]:
import sqlite3
# Step 1: leemos el dataframe
# Datos provienen de  https://www.kaggle.com/gpreda/covid-world-vaccination-progress
input_df = pd.read_csv('country_vaccinations.csv')

In [5]:
# Step 2: actualizamos el dataframe a una tabla SQL
pd_to_sqlDB(input_df,
            table_name='country_vaccinatio',
            db_name='default.db')


In [6]:
# Step 3: escribimos la consulta en una variable de tipo string
sql_query_string = """
    SELECT country, SUM(daily_vaccinations) as total_vaccinated
    FROM country_vaccinations 
    WHERE daily_vaccinations IS NOT NULL 
    GROUP BY country
    ORDER BY total_vaccinated DESC
"""

In [7]:
# Step 4: y finalmente ejecutamos la consulta 
result_df = sql_query_to_pd(sql_query_string, db_name='default.db')
result_df

,country,total_vaccinated
0,China,3.250342e+09
1,India,1.829085e+09
2,United States,5.601116e+08
3,Brazil,4.113785e+08
4,Indonesia,3.732136e+08
...,...,...
218,Falkland Islands,4.933000e+03
219,Niue,4.324000e+03
220,Montserrat,4.320000e+03
221,Tokelau,1.966000e+03


resultado del total de vacunaciones contra el covid por pais

# Updated code by Juan Cuevas Vasquez